In [2]:
import pandas as pd
from datetime import datetime, timedelta
import time
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Popup, basemaps, AwesomeIcon
from IPython.display import display
import mysql.connector
from mysql.connector import errorcode
import configparser

config= configparser.ConfigParser()
config.read('config.ini')
# Load the necessary GTFS data files
data_path = config.get("app","data_path")
stops = pd.read_csv(f"{data_path}/stops.txt")
routes = pd.read_csv(f"{data_path}/routes.txt")
trips = pd.read_csv(f"{data_path}/trips.txt")
stop_times = pd.read_csv(f"{data_path}/stop_times.txt")
calendar = pd.read_csv(f"{data_path}/calendar.txt")

#Connect to the database
db = mysql.connector.connect(
    host=config.get("app","host"),
    user=config.get("app","user"),
    password=config.get("app","password"),
    database=config.get("app","database")
)
cursor = db.cursor()
insert_buses = ("INSERT INTO buses "
               "(route_short_name, trip_id, service_id, route_id, direction_id,arrival_time,departure_time,stop_id,stop_name,stop_sequence,stop_lat,stop_lon,update_time) "
               "VALUES (%s, %s,%s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s)")
update_rows=("UPDATE buses SET OLD.route_short_name=NEW.route_short_name,")
delete_buses=("DELETE FROM buses")
# Helper function to convert time strings to HH:MM:SS format
def convert_time(time_str):
    try:
        parts = time_str.split(':')
        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
        if hours >= 24:
            hours -= 24
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    except ValueError as e:
        print(f"Error converting time: {time_str} - {e}")
        return None

# Apply the time conversion
stop_times['arrival_time'] = stop_times['arrival_time'].apply(convert_time)
stop_times['departure_time'] = stop_times['departure_time'].apply(convert_time)
stop_times = stop_times.dropna(subset=['arrival_time', 'departure_time'])

# Define the interval in seconds for updates
update_interval = 30  # 30 seconds

def get_time_input(prompt):
    while True:
        try:
            time_str = input(prompt)
            time_obj = datetime.strptime(time_str, '%H:%M:%S').time()
            return time_str
        except ValueError:
            print("Invalid time format. Please enter time in HH:MM:SS format.")

# Function to get the service IDs for a given date
def get_service_ids_for_date(date):
    day_index = date.weekday()
    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    day_column = days[day_index]
    service_ids = calendar[calendar[day_column] == 1]['service_id'].tolist()
    return service_ids
# Function to get the buses at the current time
def get_buses_at_time(target_time_str, service_ids):
    # Filter the stop_times DataFrame for the target time
    filtered_stop_times = stop_times[
        (stop_times['arrival_time'] == target_time_str) | 
        (stop_times['departure_time'] == target_time_str)
    ]
    
    if filtered_stop_times.empty:
        print("No new bus data available, retaining previous state.")
        return []

    # Merge with trips to filter by the given service IDs
    filtered_stop_times = pd.merge(filtered_stop_times, trips, on='trip_id', how='left')
    filtered_stop_times = filtered_stop_times[filtered_stop_times['service_id'].isin(service_ids)]
    
    if filtered_stop_times.empty:
        return []

    # Merge with routes and stops to get detailed bus information
    merged_df = pd.merge(filtered_stop_times, routes, on='route_id', how='left')
    merged_df = pd.merge(merged_df, stops, on='stop_id', how='left')
    
    bus_data = []
    for _, row in merged_df.iterrows():
        routeNumber = row['route_short_name']
        tripID = row['trip_id']
        routeID = row['route_id']
        stopName = row['stop_name']
        arrivalTime = row['arrival_time']
        departureTime = row['departure_time']
        serviceID = row['service_id']
        stopSequence = row['stop_sequence']
        directionID = row['direction_id']
        stopID = row['stop_id']
        stopLat = row['stop_lat']
        stopLon = row['stop_lon']
        print(
        f"Route: {routeNumber}, Trip ID: {tripID}, Stop: {stopID}, Arrival Time: {arrivalTime}, Departure Time: {departureTime}, "
        f"Stop Name: {stopName}, Direction ID: {directionID}, Stop Lat: {stopLat}, Stop Lon: {stopLon}, Route ID: {routeID}, Service ID: {serviceID}, Stop Sequence: {stopSequence}")
        bus_data.append((routeNumber, tripID, serviceID, routeID, directionID, arrivalTime, departureTime,
                         stopID, stopName, stopSequence, stopLat, stopLon))   
    return bus_data

def add_stop_markers(stops_df, mymap, stop_markers):
    for stop in stops_df.itertuples():
        marker = Marker(location=[stop.stop_lat, stop.stop_lon], draggable=False)
        popup_content = HTML()
        popup_content.value = f"Stop: {stop.stop_name}, Stop Number: {stop.stop_id}"
        popup = Popup(
            location=[stop.stop_lat, stop.stop_lon],
            child=popup_content,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        )
        marker.popup = popup
        mymap.add_layer(marker)
        stop_markers[stop.stop_id] = marker
    return stop_markers

def create_bus_marker(bus_data):
    icon = AwesomeIcon(name='bus', marker_color='red', icon_color='white')
    marker = Marker(icon=icon, location=[bus_data['stop_lat'], bus_data['stop_lon']], draggable=False)
    return marker

def update_popup(marker, bus_data):
    popup_content = HTML()
    popup_content.value = f"Route Name: {bus_data[0]}, Trip ID: {bus_data[1]}, Stop Name: {bus_data[8]}, Arrival Time: {bus_data[5]}"
    marker.popup = Popup(
        location=marker.location,
        child=popup_content,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
# Start the dynamic updating service
def dynamic_update_service(start_time_str, end_time_str):
    # Initialize start and end datetime
    today_date = datetime.now().date()
    start_time = datetime.strptime(start_time_str, '%H:%M:%S').time()
    end_time = datetime.strptime(end_time_str, '%H:%M:%S').time()
    
    start_datetime = datetime.combine(today_date, start_time)
    end_datetime = datetime.combine(today_date, end_time)
    
    # If end time is before start time, assume it is on the next day
    if end_time < start_time:
        end_datetime += timedelta(days=1)

    current_datetime = start_datetime
    previous_bus_state = {}
    markers = {}
    stop_markers = {}

    # Initialize the map
    mapcenter = (stops['stop_lat'].mean(), stops['stop_lon'].mean())
    mymap = Map(center=mapcenter, zoom=12, basemap=basemaps.OpenStreetMap.Mapnik)
    
    # Adding the stop markers and displaying the map initially
    stop_markers = add_stop_markers(stops, mymap, stop_markers)
    display(mymap)

    while current_datetime <= end_datetime:
       
        current_time_str = current_datetime.strftime('%H:%M:%S')
        print(f"\nCurrent Time: {current_time_str}")

        # Get service IDs for the current date
        current_service_ids = get_service_ids_for_date(current_datetime.date())
        buses = get_buses_at_time(current_time_str, current_service_ids)
        '''
        try:
                # Insert each bus data row into the database
                for data in buses:
                    cursor.execute(insert_buses, (*data, current_datetime))

                # Commit the transaction to save the data in the database
                db.commit()

        except mysql.connector.Error as err:
            print(f"Error: {err}")
            db.rollback()  # Rollback the transaction on error
        '''
        # Check if the buses list is empty, if so, keep the previous state
        if not buses:
            print("No new bus data available, retaining previous state.")
        else:
            # trip_id, stop_lat, stop_lon
            current_bus_state = {data[1]: (data[10], data[11]) for data in buses}
            print(">>>>> ", current_bus_state)
           
            try:
                cursor.execute(delete_buses)
            # Insert each bus data row into the database
                for data in buses:
                    cursor.execute(insert_buses, (*data, current_datetime))

            # Commit the transaction to save the data in the database
                    db.commit()

            except mysql.connector.Error as err:
                print(f"Error: {err}")
                db.rollback()  # Rollback the transaction on error

            for trip_id, (stop_lat, stop_lon) in current_bus_state.items():
                if trip_id in markers:
                    # Update existing marker location and popup
                    marker = markers[trip_id]
                    marker.location = [stop_lat, stop_lon]
                    update_popup(marker, next(bus for bus in buses if bus[1] == trip_id))
                else:
                    # Create and add new marker
                    bus_data = next(bus for bus in buses if bus[1] == trip_id)
                    new_marker = create_bus_marker({
                        'stop_lat': bus_data[10],
                        'stop_lon': bus_data[11],
                        'route_short_name': bus_data[0],
                        'trip_id': bus_data[1],
                        'stop_name': bus_data[8],
                        'arrival_time': bus_data[5]
                    })
                    update_popup(new_marker, bus_data)
                    mymap.add_layer(new_marker)
                    markers[trip_id] = new_marker

            # Update previous_bus_state with the current state for the next iteration
            
        
        # Remove buses that have completed their stop sequence
        for trip_id in list(markers.keys()):
            if trip_id not in current_bus_state:
                mymap.remove_layer(markers[trip_id])
                del markers[trip_id]
                print(f"Trip ID {trip_id} has completed its stop sequence and has been removed.")
            if trip_id in current_bus_state== trip_id in previous_bus_state:
                previous_bus_state.update(current_bus_state)    
                
        current_datetime += timedelta(seconds=update_interval)
        time.sleep(3)  # Reduce sleep time for faster testing

# Run the dynamic updating service with the specified start and end times
start_time_str = get_time_input("Enter start time (HH:MM:SS): ")
end_time_str = get_time_input("Enter end time (HH:MM:SS): ")
dynamic_update_service(start_time_str, end_time_str)


Map(center=[41.39582465849056, 27.357291164150944], controls=(ZoomControl(options=['position', 'zoom_in_text',…


Current Time: 10:00:00
Route: 2, Trip ID: 39101-0-4-67453, Stop: 39002, Arrival Time: 10:00:00, Departure Time: 10:00:00, Stop Name: ONUR MARKET, Direction ID: 0, Stop Lat: 41.4057141, Stop Lon: 27.35077, Route ID: 39101, Service ID: 4, Stop Sequence: 3
Route: 2, Trip ID: 39101-1-4-73844, Stop: 39020, Arrival Time: 10:00:00, Departure Time: 10:00:00, Stop Name: HEAS LOJMANLARI, Direction ID: 1, Stop Lat: 41.3997627, Stop Lon: 27.3763757, Route ID: 39101, Service ID: 4, Stop Sequence: 2
Route: 3, Trip ID: 39102-1-4-71108, Stop: 39301, Arrival Time: 10:00:00, Departure Time: 10:00:00, Stop Name: ALYA3, Direction ID: 1, Stop Lat: 41.3866935, Stop Lon: 27.3660363, Route ID: 39102, Service ID: 4, Stop Sequence: 1
Route: 4, Trip ID: 39103-0-4-68601, Stop: 39406, Arrival Time: 10:00:00, Departure Time: 10:00:00, Stop Name: FİDANLIK, Direction ID: 0, Stop Lat: 41.3560923, Stop Lon: 27.3257803, Route ID: 39103, Service ID: 4, Stop Sequence: 8
Route: 5, Trip ID: 39104-0-4-47056, Stop: 39254, Ar

KeyboardInterrupt: 